In [1]:
#2. Relier les bulles connectées entre elles 

import dash
from dash import html, dcc, Input, Output
import plotly.graph_objects as go
import networkx as nx
import pandas as pd



### Etude du fichier csv et compte des étiquettes 

df = pd.read_csv("Data_20250506.csv", sep=';')

etiquettes_data = df.iloc[:, 11:]  # à partir de la colonne 12, on prend tous les mots clés des études scientifiques 
etiquettes_series = pd.Series(etiquettes_data.values.ravel())

# Nettoyage des valeurs : on enlève les cases vides, les majuscules...car sinon on compterait différemment Fer et fer par exemple

etiquettes_series = etiquettes_series.dropna().astype(str).str.strip().str.lower() 

etiquettes_comptes = etiquettes_series.value_counts().reset_index()  # compte le nombre d'occurences dans toute la littérature scientifique
etiquettes_comptes.columns = ['Étiquette', 'Nombre_de_récurrence']

print(etiquettes_comptes)



#### A ce point la, certains mots sont encore compté deux fois différement  = electric vehicles et elecric vehicle par exmple !



                               Étiquette  Nombre_de_récurrence
0                                      0                 67495
1                      electric vehicles                  2675
2                        electrification                  1253
3                       electric vehicle                   558
4              battery electric vehicles                   437
...                                  ...                   ...
23195          trolleybus infrastructure                     1
23196  electrified public transportation                     1
23197                       static model                     1
23198                   computation time                     1
23199       bidirectional direct current                     1

[23200 rows x 2 columns]


In [2]:
#tokenise 
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:


text=df['AB_AI'][0]
print(text)

tokens = word_tokenize(text)

Electric vehicles have been developed with the aims of saving energy and reducing carbon dioxide emissions. Furthermore, as a single bus can transport many people, buses produce low carbon dioxide emissions per person in comparison with cars. In recent years, community buses have been introduced to ensure the mobility of senior citizens. Therefore, a low-floor vehicle was adopted as the base vehicle in our latest project. The bus was demonstrated around the center of the city. It followed a circular route of distance 7.2 kilometers, the number of bus stops on the route was 27 and the driving time was 40 minutes per circuit. This demonstration was repeated three to four times a day for 14 days. During the 86 circuits of the demonstration, a total of 1,110 passengers used the bus and the total running distance was 776 kilometers. From these results, we concluded that the quantity of carbon dioxide emissions could be reduced by 37% using our proposed bus in comparison with that of a conve

In [4]:
#Virons les stop words 

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))
print('nombre de mots de text :' , len(text))
words = word_tokenize(text)
words_filtered = []
print('nombre de mots de words :' , len(words))
# on va renvoyer une liste des mots en ne gardant "que les plus importants". On enlève par exemple les "is", "of... En somme, on ne garde que les mots clés.
for w in words:
  if w not in stopwords:
    words_filtered.append(w)



# J'enlève maintenant toute la ponctuation de ma liste de mots. 
ponctuation = ['.',',',';','?',':','!','%']
words_sans_ponctuation=[x for x in words_filtered if x not in ponctuation]


# on a beaucoup de doublons dans la liste : on les retire pour diminuer le nombre de mots 
W=list(set(words_sans_ponctuation))
print('nombre de mots de W ( sans ponctuation) :' , len(W))
# on fait un choix : on enlève les mots qui terminent par "ED" en anglais car on estime que ce sont des verbes, et non les principaux mots-clés. 


W_sans_ED = [w for w in W if not w.lower().endswith("ed")] 
print('nombre de mots de W_sans_ED :' , len(W_sans_ED))
print(W_sans_ED)

# A ce stade on a réussi à réduire le nombre de mots clés de 1026 mots à 79. Ca rste encore beaucoup trop si on veut traiter beaucoup d'articles en meme temps. 
sans_maj = [w.lower() for w in W_sans_ED]

nombre de mots de text : 1026
nombre de mots de words : 187
nombre de mots de W ( sans ponctuation) : 88
nombre de mots de W_sans_ED : 78
['citizens', 'quantity', 'circuit', 'This', '86', 'project', 'Electric', 'comparison', 'saving', 'senior', 'recent', 'The', 'city', '40', '14', 'times', 'During', 'emissions', 'ensure', 'kilometers', 'aims', 'circular', 'minutes', 'dioxide', 'It', 'bus', 'using', 'stops', 'three', 'diesel-engine', 'route', 'transport', 'days', 'From', 'reducing', 'cars', 'many', 'circuits', 'people', 'vehicle', 'time', 'base', 'four', 'low', 'per', 'number', 'carbon', 'distance', 'center', 'produce', 'day', 'energy', 'low-floor', 'single', 'years', 'person', 'community', '7.2', '776', '1,110', 'running', 'results', '37', '27', 'vehicles', 'Furthermore', 'could', 'passengers', 'mobility', 'In', 'around', 'Therefore', 'demonstration', 'conventional', 'driving', 'buses', 'latest', 'total']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# on enlève les majuscules pour pouvoir enlever les stopwords qui contenaient des majuscules 
mots_filtrés =[]
for w in sans_maj:
  if w not in stopwords:
    mots_filtrés.append(w)

print('nombre de mots de mots_filtrés :' , len(mots_filtrés))
print(mots_filtrés)

nombre de mots de mots_filtrés : 72
['citizens', 'quantity', 'circuit', '86', 'project', 'electric', 'comparison', 'saving', 'senior', 'recent', 'city', '40', '14', 'times', 'emissions', 'ensure', 'kilometers', 'aims', 'circular', 'minutes', 'dioxide', 'bus', 'using', 'stops', 'three', 'diesel-engine', 'route', 'transport', 'days', 'reducing', 'cars', 'many', 'circuits', 'people', 'vehicle', 'time', 'base', 'four', 'low', 'per', 'number', 'carbon', 'distance', 'center', 'produce', 'day', 'energy', 'low-floor', 'single', 'years', 'person', 'community', '7.2', '776', '1,110', 'running', 'results', '37', '27', 'vehicles', 'furthermore', 'could', 'passengers', 'mobility', 'around', 'therefore', 'demonstration', 'conventional', 'driving', 'buses', 'latest', 'total']


In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Mise au singulier
W_singuliers = list(set([lemmatizer.lemmatize(w, pos='n') for w in mots_filtrés]))

print('nombre de mots de W_singuliers', len(W_singuliers))
print(W_singuliers)

nombre de mots de W_singuliers 67
['776', 'ensure', 'quantity', '1,110', 'running', 'people', 'circuit', 'citizen', 'aim', 'therefore', '86', 'project', 'emission', 'minute', 'vehicle', 'time', 'base', 'year', 'circular', 'four', 'comparison', 'saving', 'senior', 'dioxide', '37', 'recent', '27', 'low', 'bus', 'using', 'city', 'could', 'per', '40', 'number', 'carbon', 'three', 'distance', 'center', 'day', 'produce', '14', 'mobility', 'energy', 'around', 'diesel-engine', 'low-floor', 'single', 'electric', 'route', 'transport', 'demonstration', 'stop', 'conventional', 'person', 'passenger', 'result', 'driving', 'community', 'latest', 'reducing', 'kilometer', '7.2', 'total', 'furthermore', 'many', 'car']


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
# j'enlève les mots de moins de 3 caractères ; 
mots_assez_long = []
for w in W_singuliers : 
    if len(w)>=3 : 
        mots_assez_long.append(w)

print('nombre de mots de mots_assez_long', len(mots_assez_long))
print(mots_assez_long)

nombre de mots de mots_assez_long 62
['776', 'ensure', 'quantity', '1,110', 'running', 'people', 'circuit', 'citizen', 'aim', 'therefore', 'project', 'emission', 'minute', 'vehicle', 'time', 'base', 'year', 'circular', 'four', 'comparison', 'saving', 'senior', 'dioxide', 'recent', 'low', 'bus', 'using', 'city', 'could', 'per', 'number', 'carbon', 'three', 'distance', 'center', 'day', 'produce', 'mobility', 'energy', 'around', 'diesel-engine', 'low-floor', 'single', 'electric', 'route', 'transport', 'demonstration', 'stop', 'conventional', 'person', 'passenger', 'result', 'driving', 'community', 'latest', 'reducing', 'kilometer', '7.2', 'total', 'furthermore', 'many', 'car']


In [12]:
import re

def est_nombre(w):
    return re.fullmatch(r"[0-9]+([.,][0-9]+)?", w) is not None

W_sans_nombres = [w for w in mots_assez_long if not est_nombre(w)]

print('nombre de mots de W_sans_nombres', len(W_sans_nombres))
print(W_sans_nombres)


nombre de mots de W_sans_nombres 59
['ensure', 'quantity', 'running', 'people', 'circuit', 'citizen', 'aim', 'therefore', 'project', 'emission', 'minute', 'vehicle', 'time', 'base', 'year', 'circular', 'four', 'comparison', 'saving', 'senior', 'dioxide', 'recent', 'low', 'bus', 'using', 'city', 'could', 'per', 'number', 'carbon', 'three', 'distance', 'center', 'day', 'produce', 'mobility', 'energy', 'around', 'diesel-engine', 'low-floor', 'single', 'electric', 'route', 'transport', 'demonstration', 'stop', 'conventional', 'person', 'passenger', 'result', 'driving', 'community', 'latest', 'reducing', 'kilometer', 'total', 'furthermore', 'many', 'car']


On a réussi à passer de 1026 mots à 59 ! Il me faudrait maintenant une IA pour enlever les mots ayant moins d'intéret en fonction du sujet pour sélectionner les mots clé importants.

In [14]:

print(len(df['AB_AI']))




7100


In [20]:
#Faisons une fonction plus rapide pour sélectionner les mots importants d'un titre d'article : 


from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('omw-1.4')

def est_nombre(w):
        return re.fullmatch(r"[0-9]+([.,][0-9]+)?", w) is not None


def mots_cles(text) : 


    words = word_tokenize(text)
    sans_maj = [w.lower() for w in words]
    words_filtered = []
    for w in sans_maj:
      if w not in stopwords:
        words_filtered.append(w)


#ponctuation 
    ponctuation = ['.',',',';','?',':','!','%']
    words_sans_ponctuation=[x for x in words_filtered if x not in ponctuation]


# doublons
    W=list(set(words_sans_ponctuation))

# sans les -ED
    W_sans_ED = [w for w in W if not w.lower().endswith("ed")] 

# singulier

    lemmatizer = WordNetLemmatizer()
    W_singuliers = list(set([lemmatizer.lemmatize(w, pos='n') for w in W_sans_ED]))

#sans les mots très courts
    mots_assez_long = []
    for w in W_singuliers : 
        if len(w)>=3 : 
            mots_assez_long.append(w)

# sans les nombres 
   
    W_sans_nombres = [w for w in mots_assez_long if not est_nombre(w)]

    return(W_sans_nombres)





[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/heloisecouet/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [71]:
L = []

#On vai maintenant parcourir tous les titres !
for i in range (len(df['AB_AI'])) : 
    L.append(mots_cles(df['AB_AI'][i]))


liste = [mot for sous_liste in L for mot in sous_liste]
print('nombre de mots dans liste :',len(liste))  
phrase = ' '.join(liste)

print('nombre de mots dans phrase : ',len(mots_cles(phrase))) 

nombre de mots dans liste : 570063
nombre de mots dans phrase :  18248


In [72]:
from collections import Counter


# Je range dans un dictionnaire le mot et son nombre d'apparition dans l'ensemble des titres, et ce par ordre décroissant d'apparition.
compteur = Counter(liste)
dico_trie = dict(compteur.most_common())



In [78]:
# Je supprime tous les mots qui apparaissent moins de 500 fois en supposant ainsi que ce sont des mots peu importants, que je n'avais pas pu supprimer avant.

compteur_filtre = {mot: nb for mot, nb in compteur.items() if nb >= 500}

compteur_filtre_trie = dict(sorted(compteur_filtre.items(), key=lambda x: x[1], reverse=True))

print('On a réduit le nombre total de "mots-clés" de :',len(liste), "à ",len(compteur_filtre_trie))
print(compteur_filtre_trie)


On a réduit le nombre total de "mots-clés" de : 570063 à  234
{'electric': 5740, 'vehicle': 5629, 'result': 3733, 'energy': 3708, 'system': 3624, 'study': 3278, 'model': 3164, 'paper': 3099, 'battery': 2693, 'charging': 2587, 'power': 2513, 'transport': 2298, 'using': 2278, 'electrification': 2077, 'different': 2068, 'emission': 2042, 'cost': 2026, 'also': 1998, 'show': 1981, 'analysis': 1940, 'however': 1855, 'impact': 1845, 'demand': 1806, 'technology': 1711, 'method': 1708, 'strategy': 1635, 'time': 1633, 'potential': 1578, 'performance': 1578, 'approach': 1546, 'data': 1524, 'use': 1479, 'fuel': 1471, 'increase': 1459, 'simulation': 1411, 'due': 1408, 'electricity': 1405, 'present': 1402, 'current': 1389, 'significant': 1387, 'high': 1379, 'carbon': 1376, 'reduce': 1364, 'network': 1310, 'sector': 1301, 'new': 1298, 'transportation': 1294, 'case': 1292, 'two': 1289, 'research': 1285, 'solution': 1282, 'scenario': 1260, 'operation': 1244, 'one': 1243, 'consumption': 1242, 'optimizat

## En pure comparaison, j'ai demandé à chatGPT de me choisir les mots importants dans la liste L avec les 570000 mots : voici le résultat : 

In [79]:
dico_filtré = {
    'electric': 5740,
    'vehicle': 5629,
    'energy': 3708,
    'system': 3624,
    'study': 3278,
    'model': 3164,
    'battery': 2693,
    'charging': 2587,
    'power': 2513,
    'transport': 2298,
    'electrification': 2077,
    'emission': 2042,
    'cost': 2026,
    'analysis': 1940,
    'impact': 1845,
    'demand': 1806,
    'technology': 1711,
    'method': 1708,
    'strategy': 1635,
    'performance': 1578,
    'approach': 1546,
    'data': 1524,
    'fuel': 1471,
    'simulation': 1411,
    'electricity': 1405,
    'carbon': 1376,
    'reduce': 1364,
    'network': 1310,
    'sector': 1301,
    'transportation': 1294,
    'research': 1285,
    'solution': 1282,
    'operation': 1244,
    'consumption': 1242,
    'optimization': 1241,
    'grid': 1230,
    'efficiency': 1225,
    'infrastructure': 1205,
    'development': 1172,
    'environmental': 1171,
    'management': 1152,
    'reduction': 1133,
    'optimal': 1131,
    'range': 1126,
    'hybrid': 1119,
    'renewable': 1110,
    'policy': 1100,
    'driving': 1091,
    'source': 1069,
    'algorithm': 1069,
    'station': 1043,
    'capacity': 1020,
    'design': 978,
    'storage': 978,
    'distribution': 973,
    'process': 940,
    'dynamic': 934,
    'application': 917,
    'generation': 874,
    'charge': 871,
    'economic': 863,
    'adoption': 846,
    'framework': 844,
    'conventional': 827,
    'fleet': 819,
    'public': 818,
    'urban': 812,
    'dioxide': 809,
    'greenhouse': 801,
    'sustainable': 792,
    'parameter': 781,
    'transition': 760,
    'behavior': 760,
    'key': 759,
    'mobility': 725,
    'cycle': 721,
    'supply': 712,
    'characteristic': 704,
    'service': 699,
    'reducing': 679,
    'engine': 677,
    'operating': 664,
    'climate': 654,
    'cell': 635,
    'alternative': 633,
    'planning': 630,
    'internal': 628,
    'modeling': 626,
    'efficient': 621,
    'mode': 619,
    'industry': 618,
    'air': 615,
    'objective': 609,
    'combustion': 608,
    'environment': 605,
    'production': 602,
    'effective': 591,
    'integration': 559,
    'electrical': 545,
    'bus': 538,
    'smart': 502,
}


In [80]:
print(len(dico_filtré)) 

101


On passe de 234 mots avec ma méthode à 101 mots avec chatGPT. Sur 570 000 mots d'origine, je considère que ma méthode est plutôt satisfaisante, surtout que les 101 mots sont bien compris 

In [89]:
# On veut maintenant calculer des poids d'importance des mots pour qu'on puisse attribuer à chaque mot un rayon plus ou moins grand. 
# On fait une echelle pour avoir des rayons compris entre 10 et 100. 
def scale(freq, min_freq, max_freq, min_scale=10, max_scale=100):
    return min_scale + (freq - min_freq) * (max_scale - min_scale) / (max_freq - min_freq)

min_freq = min(dico_filtré.values())
max_freq = max(dico_filtré.values())

rayon_mots = {
    mot: round(scale(freq, min_freq, max_freq), 2)
    for mot, freq in dico_filtré.items()
}

print(rayon_mots)

{'electric': 100.0, 'vehicle': 98.09, 'energy': 65.09, 'system': 63.64, 'study': 57.7, 'model': 55.74, 'battery': 47.65, 'charging': 45.82, 'power': 44.55, 'transport': 40.86, 'electrification': 37.06, 'emission': 36.46, 'cost': 36.19, 'analysis': 34.71, 'impact': 33.08, 'demand': 32.41, 'technology': 30.77, 'method': 30.72, 'strategy': 29.47, 'performance': 28.49, 'approach': 27.94, 'data': 27.56, 'fuel': 26.65, 'simulation': 25.62, 'electricity': 25.52, 'carbon': 25.02, 'reduce': 24.81, 'network': 23.88, 'sector': 23.73, 'transportation': 23.61, 'research': 23.45, 'solution': 23.4, 'operation': 22.75, 'consumption': 22.71, 'optimization': 22.7, 'grid': 22.51, 'efficiency': 22.42, 'infrastructure': 22.08, 'development': 21.51, 'environmental': 21.49, 'management': 21.17, 'reduction': 20.84, 'optimal': 20.81, 'range': 20.72, 'hybrid': 20.6, 'renewable': 20.45, 'policy': 20.27, 'driving': 20.12, 'source': 19.74, 'algorithm': 19.74, 'station': 19.3, 'capacity': 18.9, 'design': 18.18, 'st

In [95]:
from collections import defaultdict
import itertools
import pandas as pd


tous_mots = list(dico_filtré)

cooc_matrice = defaultdict(lambda: defaultdict(int))

# On parcourt chaque sous-liste de mots (chaque titre)
for mots in L:
    mots_filtres = [m for m in set(mots) if m in tous_mots] 
    for w1, w2 in itertools.combinations(mots_filtres, 2):
        cooc_matrice[w1][w2] += 1
        cooc_matrice[w2][w1] += 1  #comme on aura une matrice symétrique, on essaie de réduire la complexité en ne parcourant pas deux fois
        
df_cooc = pd.DataFrame(cooc_matrice, index=tous_mots, columns=tous_mots).fillna(0).astype(int)


print(df_cooc.shape)  
print(df_cooc.head())


(101, 101)
          electric  vehicle  energy  system  study  model  battery  charging  \
electric         0     5249    2970    2937   2722   2668     2454      2469   
vehicle       5249        0    2916    2854   2654   2613     2359      2355   
energy        2970     2916       0    2221   1699   1673     1551      1346   
system        2937     2854    2221       0   1660   1662     1449      1384   
study         2722     2654    1699    1660      0   1552     1266      1254   

          power  transport  ...  air  objective  combustion  environment  \
electric   2124       1555  ...  486        534         542          493   
vehicle    2022       1465  ...  490        521         555          485   
energy     1669       1312  ...  333        378         321          352   
system     1679       1191  ...  324        356         226          326   
study      1078       1059  ...  281        302         302          280   

          production  effective  integration  elect

In [105]:
### Liens entre les bulles de mots 

liens= []

for a, b in itertools.combinations(df_cooc.columns, 2):
    if df_cooc.loc[a, b] > 1000:  # on choisi de garder que les mots que reviennent ensemble de les titres plus de 1000 fois.
        liens.append((a, b))

print(f"{len(liens)} liens trouvés avec cooccurrence > 1000")
print(liens)  # aperçu des premières paires



117 liens trouvés avec cooccurrence > 1000
[('electric', 'vehicle'), ('electric', 'energy'), ('electric', 'system'), ('electric', 'study'), ('electric', 'model'), ('electric', 'battery'), ('electric', 'charging'), ('electric', 'power'), ('electric', 'transport'), ('electric', 'electrification'), ('electric', 'emission'), ('electric', 'cost'), ('electric', 'analysis'), ('electric', 'impact'), ('electric', 'demand'), ('electric', 'technology'), ('electric', 'method'), ('electric', 'strategy'), ('electric', 'performance'), ('electric', 'approach'), ('electric', 'data'), ('electric', 'fuel'), ('electric', 'simulation'), ('electric', 'electricity'), ('electric', 'carbon'), ('electric', 'reduce'), ('electric', 'network'), ('electric', 'transportation'), ('electric', 'research'), ('electric', 'solution'), ('electric', 'operation'), ('electric', 'consumption'), ('electric', 'optimization'), ('electric', 'grid'), ('electric', 'infrastructure'), ('electric', 'management'), ('electric', 'optimal'